In [ ]:
!pip install unidecode
!pip install textattack

     |████████████████████████████████| 245kB 6.4MB/s 
     |████████████████████████████████| 358kB 6.4MB/s 
     |████████████████████████████████| 337kB 17.5MB/s 
     |████████████████████████████████| 204kB 20.6MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 778kB 19.1MB/s 
     |████████████████████████████████| 102kB 8.1MB/s 
     |████████████████████████████████| 2.1MB 31.9MB/s 
     |████████████████████████████████| 20.1MB 8.9MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 
     |████████████████████████████████| 798kB 33.8MB/s 
     |████████████████████████████████| 19.7MB 1.6MB/s 
     |████████████████████████████████| 983kB 28.8MB/s 
     |████████████████████████████████| 1.2MB 31.1MB/s 
     |████████████████████████████████| 112kB 42.5MB/s 
     |████████████████████████████████| 245kB 40.9MB/s 
     |████████████████████████████████| 3.3MB 38.0MB/s 
     |████████████████████████████████| 901kB 21.1MB/s 

In [1]:
from sklearn.datasets import fetch_20newsgroups
from bs4 import BeautifulSoup
import re
import unidecode
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train',
                                  categories=categories,
                                  shuffle=True,
                                  remove=('headers', 'footers', 'quotes'),
                                  random_state=42)
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [6]:
data = twenty_train.data
target = twenty_train.target

In [7]:
def clean_data(text, max_len=300):
    soup = BeautifulSoup(text)
    html_pattern = re.compile('<.*?>')
    text = html_pattern.sub(r' ', soup.text)
    text = unidecode.unidecode(text)
    text = re.sub('[^A-Za-z0-9.]+', ' ', text)
    text = text.lower()
    text = text[:max_len]
    return text

In [8]:
data = [clean_data(txt) for txt in data]

In [12]:
from textattack.augmentation import EasyDataAugmenter
import random
eda_augmenter = EasyDataAugmenter(pct_words_to_swap=0.1, transformations_per_example=1)

def augment_data_textattack(data, target, augmenter):
  augmented_data_eda = []
  augmented_label_eda = []
  for text, label in zip(data, target):
    if random.randint(0,2) != 1:
      augmented_data_eda.append(text)
      augmented_label_eda.append(label)
      continue
    aug_list = augmenter.augment(text)

    augmented_data_eda.append(text)
    augmented_label_eda.append(label)

    augmented_data_eda.extend(aug_list)
    augmented_label_eda.extend([label]*len(aug_list))
  return augmented_data_eda, augmented_label_eda




In [ ]:
augmented_data_eda, augmented_label_eda = augment_data_textattack(data, target, eda_augmenter)

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(augmented_data_eda)
X_train_counts.shape

NameError: ignored

In [13]:
from textattack.augmentation import CheckListAugmenter
checklist_augmenter = CheckListAugmenter(pct_words_to_swap=0.1, transformations_per_example=1)

augmented_data_checklist, augmented_label_checklist = augment_data_textattack(data, target, checklist_augmenter)

2021-04-22 17:39:53,107 loading file /root/.flair/models/en-ner-conll03-v0.4.pt


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(augmented_data_checklist)
X_train_counts.shape

(3039, 12148)

In [17]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score


mnb = MultinomialNB()

In [18]:
print("Mean Accuracy: {:.2}".format(cross_val_score(mnb, X_train_counts, augmented_label_checklist, cv=5).mean()))

Mean Accuracy: 0.82
